# COMP0142

# Assignment 2: Classification with Naive Bayes & Logistic Regression

## Introduction

In this assignment, we will implement multiclass classification using Logistic Regression and Naive Bayes via the Scikit-learn library.

In Part A of this assignment, we are going to implement Logistic Regression. First, we are going to examine how data scaling affects the performance of the classifier: We will produce a classification report and plot a confusion matrix. We are then going to use cross validation to more reliably compare the performance of the models.

In Part B of this assignment, we are going to implement multiclass and Bernoulli Naive Bayes. We are going to perform feature selection and analyse the performance of the model. Finally, we are also going to look at decision boundaries.

We are going to work with two datasets:
- The 'Gene expression cancer RNA-Seq' dataset 
- The 'Zoo' dataset.

## Guidelines

- The structure of the code is given to you and you will need to fill in the parts corresponding to each question. 
- You will have to submit the completed notebook in the Jupyter notebook format: `.ipynb`.
- Do not modify/erase other parts of the code if you have not been given specific instructions to do so.
- When you are asked to insert code, do so between the areas which begin:
  
  ````
  ##########################################################
  # TO_DO
  
  #[your code here]
  ````
   
   And which end:
   
  ````
  # /TO_DO
  ##########################################################
  ````


- When you are asked to comment on the results you should give clear and comprehensible explanations. Write the comments in a 'Code Cell' with a sign `#` at the beginning of each row, and in the areas which begin:

  `# [INSERT YOUR ANSWER HERE]`


- Some of the code you are asked to write may generate **warning messages**. Some boilerplate code is included to suppress these warnings, but you may wish to comment it out while you are working so that you don't missing useful information.

## Imports

Please do not change the cell below, you will see a number of imports. All these packages are relevant for the assignment and it is important that you get used to them. You can find more information about them in their respective documentation. The most relevant package for this assignment is Scikit-learn:

https://scikit-learn.org/stable/

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

# Standard python libraries for data and visualisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
%matplotlib inline
import seaborn as sns

# SciKit Learn python ML Library
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB

# Import error metric
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Import library for handling warnings
import warnings

: 

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

# Functions to use
# Decision boundary plotting 
def plot_predictions(X, y, clf):
    h = .02  # step size in the mesh
 
    # create a mesh to plot in
    x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
    y_min, y_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    plt.figure(figsize=(8,6))

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)

    # Plot also the training points
    pylab.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, cmap=plt.cm.coolwarm)
    plt.xlabel(list(X.head(0))[0])
    plt.ylabel(list(X.head(0))[1])
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(np.arange(min(X.iloc[:, 0]), max(X.iloc[:, 0])+1, 1.0))
    plt.yticks(np.arange(min(X.iloc[:, 1]), max(X.iloc[:, 1])+1, 1.0))
    plt.title(clf)

    plt.show
    
# confusion matrix plotting
def plot_conf_matrix(conf_matrix):
    plt.figure(figsize=(5,5))
    sns.heatmap(conf_matrix, annot=True, cmap="YlGnBu" ,fmt='g')    
    plt.ylabel('Actual label');
    plt.xlabel('Predicted label');

: 

# Part A: Logistic Regression [50 marks]

## Gene expression dataset
This dataset contains gene expression data from patients diagnosed with one of tumor types: BRCA, KIRC, COAD, LUAD and PRAD. Each feature corresponds to a different gene. 

Dataset location: https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq

Number of instances: 801

Number of features: 20531   

All of these parameters are real-valued continuous. To reduce computation time, we are going to work with the first 200 features.

## Load dataset
Please save the `data.csv` and `labels.csv` files included in the assignment zip file, which contain this data, and change the paths below to the paths leading to the location of your downloaded files. You may want to use os.chdir to change directory. 

In [ ]:
# PLEASE CHANGE THE FILE PATHS AS NEEDED

file_path_data = "data.csv"
file_path_labels = "labels.csv"

: 

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

# read the file with pandas.read_csv
X = pd.read_csv(file_path_data, usecols=[*range(1, 201)])
y = pd.read_csv(file_path_labels, usecols=[1]).values.ravel()

label_list = ["BRCA", "KIRC", "COAD", "LUAD", "PRAD"]

: 

## Data analysis and pre-processing 
Below, we will generate histograms of the first 12 'Gene expression cancer RNA-Seq' dataset features.

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

figs, axs = plt.subplots(3, 4, figsize=(12, 10))
axs = axs.ravel()

for counter in range(12):
    col = X.columns[counter]
    axs[counter].hist(X[col], bins=20)
    axs[counter].set_title(col)

: 

Regularisation makes the classifier dependent on the scale of the features. 

We are going to scale the features and compare the performance of Logistic Regression on unscaled and scaled dataset. 

### _Question 1 [10 marks]_ 

### _a) [3 marks]_ 
- Use `StandardScaler()` to scale the data. Save the result to a new variable (do not overwrite X).

In [ ]:
#######################################################
# TO_DO

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


#/TO_DO
#######################################################

: 

### _b) [3 marks]_
-  Explain how the `StandardScaler()` function changes the data, (in particular its mean and variance)? (**Hint:** You can re-run the code from the section **Data analysis and pre-processing** in order to visualise scaled values.)

In [ ]:
# [INSERT YOUR ANSWER HERE]

'''
Standard scaler works by normalising the distribution of data to a mean = 0 and a standard deviation of 1. As the distribution itself is normalised,
the structure of data is maintained as relationships between the individual data points are preserved, despite the data values themselves change.
The standard score of a sample x is calculated as: z = (x - u) / s where 'u' is the mean and 's' is the standard deviation.

First you calculate the mean value for each feature in the dataset and then subtract this mean from every individual data point within that feature.
This new mean of the standardised data is centered around 0.

Next, you divide the centred data by the standard deviation to scale the variance of each feature to 1. This is because variance is defined as the 
average of the squared differences from the mean, and by dividing by the standard deviation, you're normalizing these squared differences. The 
standardised data achieve a unit variance of 1. Since the data was centred around zero in the first step, and is now scaled by the standard deviation, 
the result is a feature with a variance of 1. It makes sense as variance can be calculated by the square of standard deviation (in a standard data 
distribution), the variance is therefore 1.
'''



: 

### _c) [4 marks]_
- `LogisticRegression()` uses $\ell_2$ regularisation as default. Briefly explain the effect of such a regulariser. Furthermore, briefly explain why data scaling might be a useful pre-processing step before the application of such a regulariser.

In [ ]:
# [INSERT YOUR ANSWER HERE]
'''
Regularisation provides a penalising term to a model of increasing complexity. It controls the amount of shrinkage, pushing weight coefficients towards 
0 in order to reduce the loss function. 

The effect of a regulariser such as the l2 norm controls the effect of overfitting. For example, in the case of perfectly separable data, the gradient 
descent approach may provide an infinite solution and therefore may need a regularised term for it to be finite and unique when calculating MLE. 
It penalises large coefficients and therefore the model is discouraged from depending too much on any one feature and lowers the complexity of the model as a whole.

Ridge regulariser L2 also has the benefit of ensuring that gradient descent leads to a globally unique solution for any input data. It means that
the model is more robust and therefore is less likely to converge to overfitting solutions, making the model more resistant to minor changes in the training data.

Data scaling might be a useful pre-processing step before applying a regulariser because it normalises features to a similar scale. Parameters
will have values in the same range, which eliminates potential biases and distortions that maybe caused by having test and training data of different scales,
and therefore remove improper/disproportionate penalisation when parameters are large (this may be the case prior to data scaling). It would mean that each feature
can contribute equally to the model's predictions thanks to this uniform approach. Improper penalisation leads to increased generalisability of models, which is undesired 
when we want to optimise a model for a particular dataset.

Another benefit of scaling is that it ensures smoother optimisation through improved convergence to minima. Intuitively, processes such as gradient descent is sped up in smaller 
ranges of values compared to values of a larger range.
'''

: 

### Classifier performance analysis

A Confusion Matrix is a table used for the evaluation of classification models. The x axis represents predicted labels while the y axis represents actual labels. Each cell indicates the sum of instances assigned to a particular combination of these labels. Diagonal values represents correctly classified instances.  

### _Question 2 [20 marks]_

### _a) [5 marks]_ 
- Create training and testing datasets for the unscaled and scaled data (set `random_state=42` and `train_size=0.7` when making your split).

In [ ]:
lg = LogisticRegression(solver = "lbfgs", multi_class = "multinomial", max_iter = 5000)
lg_scaled = LogisticRegression(solver = "lbfgs", multi_class = "multinomial", max_iter = 5000)

#######################################################
# TO_DO

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled, y, train_size=0.7, random_state=42)


#/TO_DO
#######################################################

: 

### _b) [5 marks]_ 
- Fit `LogisticRegression()` to the unscaled and scaled data.

In [ ]:
with warnings.catch_warnings():
    
    ########################################################
    # TO_DO

    # NB: you may wish to comment out this filter
    warnings.simplefilter("ignore")
    
    lg.fit(X_train, y_train)
    lg_scaled.fit(X_train_scaled, y_train_scaled)


    #/TO_DO
    #######################################################

: 

### _c) [5 marks]_ 
- Plot confusion matrices for the scaled and unscaled data using Scikit-learn `confusion_matrix()` and the `plot_conf_matrix()` function defined for you at the beginning of the notebook.

In [ ]:
#######################################################
# TO_DO

y_pred = lg.predict(X_test)
y_pred_scaled = lg_scaled.predict(X_test_scaled)

conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_scaled = confusion_matrix(y_test_scaled, y_pred_scaled)

plot_conf_matrix(conf_matrix)
plot_conf_matrix(conf_matrix_scaled)
                 

#/TO_DO
#######################################################

: 

### _d) [5 marks]_ 
- Print a classification report using scikit-learn `classification_report()` function. You can use `target_names = label_list` to include labels.

In [ ]:
with warnings.catch_warnings():
    
    ########################################################
    # TO_DO

    # NB: you may wish to comment out this filter
    warnings.simplefilter("ignore")

    print("Classification Report (Unscaled Data):")
    print(classification_report(y_test, y_pred, target_names=["BRCA", "KIRC", "COAD", "LUAD", "PRAD"]))

    print("\nClassification Report (Scaled Data):")
    print(classification_report(y_test_scaled, y_pred_scaled, target_names=["BRCA", "KIRC", "COAD", "LUAD", "PRAD"]))


    #/TO_DO
    #######################################################

: 

### Cross validation
In Scikit-learn, `StratifiedKFold()` splits the data into $k$ different folds.  
`cross_val_score()` then uses these folds to run the classifier multiple times and collect multiple accuracy scores.   

### _Question 3 [20 marks]_

### _a) [5 marks]_ 
- Split data using `StratifiedKFold()`. Set `n_splits = 10`, `shuffle = True`, and `random_state=42`.

In [ ]:
with warnings.catch_warnings():
    
    ########################################################
    # TO_DO

    # NB: you may wish to comment out this filter
    warnings.simplefilter("ignore")

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    for train_index, test_index in skf.split(X, y):
       X_train, X_test = X.iloc[train_index], X.iloc[test_index]
       y_train, y_test = y[train_index], y[test_index]

    
    #/TO_DO
    ########################################################

: 

### _b) [5 marks]_ 
- Calculate cross validation scores using `cross_val_score()`. Call the variables storing these scores `lg_scores` and `lg_scaled_scores` (for consistency with plotting done for you in the subsequent section). (**Hint:** `cv` is equal to the output of `StratifiedKFold()`.)

In [ ]:
with warnings.catch_warnings():
    
    ########################################################
    # TO_DO

    # NB: you may wish to comment out this filter
    warnings.simplefilter("ignore")
    
    lg_scores = cross_val_score(lg, X, y, cv=skf)
    lg_scaled_scores = cross_val_score(lg_scaled, X_scaled, y, cv=skf)
    
    #/TO_DO
    ########################################################

: 

### _c) [5 marks]_ 
- Calculate and print the mean of the scores.

In [ ]:
with warnings.catch_warnings():
    
    ########################################################
    # TO_DO

    # NB: you may wish to comment out this filter
    warnings.simplefilter("ignore")
    
    print(f"Mean cross-validation score (Unscaled Data): {lg_scores.mean()}")
    print(f"Mean cross-validation score (Scaled Data): {lg_scaled_scores.mean()}")

    
    #/TO_DO
    ########################################################

: 

### _d) [5 marks]_ 
- Unlike vanilla `KFold()`, `StratifiedKFold()` aims to preserve the proportion of examples belonging to each class in each split. Does `StratifiedKFold()` make each data split balanced if the whole dataset is not balanced?

In [ ]:
# [INSERT YOUR ANSWER HERE]
'''
If the dataset is balanced, StratifiedFold() will keep the balance across each fold with each class being equally represented. However if the dataset is not 
balanced, then the function does not balance the number of examples from each class. It ensures that the proportion of each class within each fold mirrors the 
overall dataset's proportions rather than balancing the number of samples from each class in each fold. This approach is particularly beneficial for dealing with 
imbalanced datasets since it eliminates the danger of removing minority classes from some folds, which is a potential concern with vanilla KFold() splitting.
'''

: 

We can visualise the scores using a box plot. It highlights the lower and upper quartiles, and "whiskers" showing the extent of the scores.  

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

plt.figure(figsize=(8, 4))
plt.plot([1]*10, lg_scores, ".")
plt.plot([2]*10, lg_scaled_scores, ".")
plt.boxplot([lg_scores, lg_scaled_scores], labels=("logistic regression","logistic regression w/ scaling"))
plt.ylabel("Accuracy", fontsize=14)
plt.show()

: 

# Part B: Naive Bayes [50 marks]

Please note that we are still working with the 'Gene expression cancer RNA-Seq' dataset loaded in Part A.  

### Removing correlated features
Feature independence is an assumption of Naive Bayes. Naive Bayes is particularly sensitive to feature correlations which can lead to overfitting. Based on data alone, we cannot test if features are truly independent, but we can exclude correlated features.
Below, we test if features are correlated.

### _Question 4 [10 marks]_
Drop features with correlation above 0.75.

**Hint:** see what `to_drop` returns, then use it as an argument in the *pandas* `drop()` function with `axis=1` and `inplace=True`.

In [ ]:
# Create correlation matrix
corr_matrix = X.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.75
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]

#######################################################
# TO_DO

X.drop(columns=to_drop, axis=1, inplace=True)
# the code here uses pd drop to drop the data that are correlated. 
# Here an arbitrary value of 0.75 was used to show correlation in these genes


#/TO_DO
#######################################################

print("Correlated features dropped: ")
print(*to_drop, sep = ", ") 

: 

### Recursive feature elimination
Lets go further and select the 5 most important features. Recursive Feature Elimination (RFE) is designed to select features by recursively considering smaller and smaller sets of features.  

### _Question 5 [10 marks]_

### _a) [5 marks]_ 
- Use the `RFE()` function in Scikit-learn to select features. (**Hint:** The `MultinomialNB` and `BernoulliNB` models do not have a `coef_` attribute, but the attribute `feature_log_prob_` can serve a similar purpose. Check the Scikit-learn documentation and examples for more information.)

### _b) [5 marks]_ 
- After selecting features to eliminate, use the `support_` attribute as a mask to select the right columns.

In [ ]:
nb = MultinomialNB()

#######################################################
# TO_DO
# [your code here]
# im pretty sure the normal approach is to split the data into test and train
# then fit the data to the classification
from operator import attrgetter

nb.fit(X, y) 
rfe = RFE(estimator=nb, 
          n_features_to_select=5, step=1,
          importance_getter=attrgetter("feature_log_prob_")) # here i select the 5 most important features
rfe.fit(X, y)
selected_features = rfe.support_
selected_feature_indices = np.where(selected_features)[0] # obtain indices of selected features
selected_features = X.columns[selected_feature_indices] # get column names of selected features
print(f"Selected features: {', '.join(map(str, selected_features))}")

#/TO_DO
#######################################################

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
nb.fit(X_train, y_train)
nb_predict = nb.predict(X_test)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    print(classification_report(y_test, nb_predict, target_names=label_list))

nb_confusion = confusion_matrix(y_test, nb_predict)
plot_conf_matrix(nb_confusion)

: 

We are now going to switch to a different dataset.
### Zoo dataset 
This is a simple dataset which classifies animals into 7 categories. 

Dataset location: https://archive.ics.uci.edu/ml/datasets/Zoo  

Number of instances: 210

Number of features: 17

Attribute Information:  
1. animal name: Unique for each instance   
2. hair:	Boolean   
3. feathers:	Boolean   
4. eggs:	Boolean   
5. milk:	Boolean   
6. airborne:	Boolean   
7. aquatic:	Boolean   
8. predator:	Boolean   
9. toothed:	Boolean   
10. backbone:	Boolean   
11. breathes:	Boolean   
12. venomous:	Boolean   
13. fins:	Boolean   
14. legs:	Numeric (set of values: {0,2,4,5,6,8})   
15. tail:	Boolean   
16. domestic:	Boolean   
17. catsize:	Boolean   
18. type:	Numeric (integer values in range [1,7])  

All of these parameters are discrete-valued.
  
### Load dataset
Please save the 'zoo.csv' file included in the assignement zip file, which contains a subset of this data, and change the paths below to the paths leading  to the location of your downloaded files. You may want to use `os.chdir` to change directory.

In [ ]:
# PLEASE CHANGE THE FILE PATHS AS NEEDED

file_path_data_zoo = "zoo.csv"

: 

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

# read the file with pandas.read_csv
data_zoo = pd.read_csv(file_path_data_zoo)

# because the file does not contain header information, we manually add headers of the dataset
data_zoo.columns = ["animal name", "hair", "feathers","eggs", 
                "milk", "airborne","aquatic", "predator", "toothed", "backbone", 
                    "breathless", "venomous", "fins", "legs", "tail", "domestic", "catsize", "class"]

: 

We assign columns 2 to 18 (everything other than animal name and class) to variable `X_zoo`. Remember that indexing starts at 0.  
We assign the "class" column to variable `y`.  
We then split `X` and `y` into train and test datasets.

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

X_zoo = data_zoo.iloc[:,1:17]
y_zoo = data_zoo["class"]
X_zoo_train, X_zoo_test, y_zoo_train, y_zoo_test = train_test_split(X_zoo, y_zoo, test_size=0.2, random_state=42)

: 

Below, we create a test dataset which contains only animals with 4 or fewer legs. 

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

X_drop_train = X_zoo_train.drop(X_zoo_train[X_zoo_train["legs"]>4].index)
y_drop_train = y_zoo_train.drop(X_zoo_train[X_zoo_train["legs"]>4].index)

: 

We create an instance of a multinomial Naive Bayes classifier. We train `nb` on `X_drop_train`, and test it on `X_zoo_test`.  
You should get a warning message suggesting that the value of alpha is automatically overwritten. (Note that this message is not suppressed.)

In [ ]:
#PLEASE DO NOT CHANGE THIS CELL

nb = MultinomialNB(alpha =0)
nb.fit(X_drop_train, y_drop_train)
nb_predict_train = nb.predict(X_drop_train)
nb_predict_test = nb.predict(X_zoo_test)
print(accuracy_score(nb_predict_train, y_drop_train))
print(accuracy_score(nb_predict_test, y_zoo_test))

: 

### _Question 6 [10 marks]_   
Please comment on what the alpha parameter does.
**Hints:**  
- Think what Naive Bayes does when encountering a discrete feature value of which is absent in the train dataset but which is present in the test dataset. What probability estimate would be associated with it?  
- See: Scikit-learn `MultinomialNB` documentation, in particular the description of the alpha parameter https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html  

In [ ]:
### [INSERT YOUR ANSWER HERE]
'''
The alpha parameter demonstrates the stength of smoothing in NB in a process called additive
(laplace) smoothing, mathematically derived from the Dirichelet distributions.

The parameter prevents zero probabilities, which can be problematic in this scenario
when discrete feature values are absent in the train dataset, whilst being present in the test 
dataset. This is because NB assigns a 0 to a probability estimate such as MLE (maximum 
log-likelihood estimator). This case of overfitting leads to Py(k|x) being undefined. 

Additive smoothing works by adding the alpha parameter uniformly adding instances to the data. 
The alpha parameter is added to the numerator of the probability estimate MLE 
(number of training points X in animal "class" + alpha parameter) 
as well as the denominator of the MLE
(number of training points in animal "class" + alpha parameter x number of features in data).
'''
# 

: 

We are now going  to repeat the above process using Naive Bayes for features with the Bernoulli distribution (`BernoulliNB()` in scikit-learn). We are particularly interested in interpreting decision boundaries of Bernoulli Naive Bayes.

### _Question 7 [20 marks]_

### _a) [2 marks]_ 
- Use recursive feature elimination (RFE) is to select only 2 features (**Note:** in real life cases, you are likely to use this approach to select multiple rather than just two features. In this case we are asking you to analyse the outcome, and 2 features allows us to visualise the decision boundary more easily.)

### _b) [1 mark]_ 
- After selecting the features to eliminate, use the `support_` attribute as a mask to select the right columns. 

In [ ]:
bnb = BernoulliNB()

#######################################################
# TO_DO
#[your code here]
from operator import attrgetter

bnb.fit(X_zoo,y_zoo)
estimator = bnb
rfe_zoo = RFE(estimator, 
          n_features_to_select=2, step=1,
          importance_getter=attrgetter("feature_log_prob_")) # the 2 most important features are selected
rfe_zoo.fit(X_zoo,y_zoo)

zoo_selected_features = rfe_zoo.support_
zoo_selected_features_indices = np.where(zoo_selected_features)[0] # obtain indices of selected features
X_zoo_selected_features = X_zoo.columns[zoo_selected_features_indices] # get column names of selected features
print(f"Selected features: {', '.join(map(str, X_zoo_selected_features))}")

#/TO_DO
#######################################################

: 

### _c) [2 marks]_ 
- Split data into train and test sets (set `random_state=42`).

### _d) [1 mark]_ 
- Fit the model.

In [ ]:
#######################################################
# TO_DO
#[your code here]
X_zoo_train, X_zoo_test, y_zoo_train, y_zoo_test = train_test_split(X_zoo, y_zoo, test_size=0.2, random_state=42)
bnb.fit(X_zoo_train, y_zoo_train)

#/TO_DO
##########################################################

: 

### _e) [2 marks]_ 
- Create a Confusion Matrix using the `plot_conf_matrix()`

In [ ]:
#######################################################
# TO_DO
#[your code here]
bnb_predict = bnb.predict(X_zoo_test)
bnb_confusion = confusion_matrix(y_zoo_test, bnb_predict)
plot_conf_matrix(bnb_confusion)

#/TO_DO
##########################################################

: 

### _f) [2 marks]_ 
- Plot decision boundaries using the `plot_predictions()` function.

In [ ]:
with warnings.catch_warnings():
    
    ########################################################
    # TO_DO

    # NB: you may wish to comment out this filter
    warnings.simplefilter("ignore")
    
    #[your code here]
    print(X_zoo_train.shape)
    # output provided was (80,16)
    
    X_zoo_train_selected_features = X_zoo_train.iloc[:, zoo_selected_features_indices]
    
    # verify shape of X_zoo_train_selected_features
    print(X_zoo_train_selected_features.shape)
    # output provided was (80,2)
    bnb.fit(X_zoo_train_selected_features, y_zoo_train)

    plot_predictions(X_zoo_train_selected_features, y_zoo_train, bnb)
    

    #/TO_DO
##########################################################

: 

### _g) [10 marks]_ 
- Interpret the decision boundaries: Recall the shapes of decision boundaries you have seen in classes - were they straight and crossing at right angles? Why is this the case when using `BernoulliNB()`?

In [ ]:
### [INSERT YOUR ANSWER HERE]
'''
The decision boundaries shown above are straight lines and cross at right angles. The 2D decision boundaries formed from different classification 
techniques are not parallel to the axes such as in logistic regression or are curved/quadratic in some generative classifications. These shapes 
arise due to the conditional probabilities of these features, more particularly the diverse range of interactions these data values have.

The decision boundaries above are parallel to the axes because of the assumption of binary values, which are then modelled using Bernoulli 
distributions. The boundaries are much simpler due to values being confined to either a 0 or a 1. It makes sense that it is modelled this 
way as the data provided are booleans that take up either True or False values. Since these boundaries are straight lines parallel to the 
axes, we know that the decisions are based on whether a particular feature is above or below a certain threshold. 

Assuming that 0 is false and 1 is true, above shows a 3 points on the graph. The dataset only shows 3 potential options that animals either:
1. does not produce milk and does not have feathers
2. produces milk but does not have feathers
3. does not produce milk but does have feathers.

The boundaries does not suggest the possibility of an animal that both produces milk and has feathers.

'''
# 

: 

: 